In [4]:
import random
import toyplot

In [5]:
def get_kmer_count_from_sequence(sequence, k=3, cyclic=True):
    """
    Returns dictionary with keys representing all possible kmers in a sequence
    and values counting their occurrence in the sequence.
    """
    # dict to store kmers
    kmers = {}
    
    # count how many times each occurred in this sequence (treated as cyclic)
    for i in range(0, len(sequence)):
        kmer = sequence[i:i + k]
        
        # for cyclic sequence get kmers that wrap from end to beginning
        length = len(kmer)
        if cyclic:
            if len(kmer) != k:
                kmer += sequence[:(k - length)]
        
        # if not cyclic then skip kmers at end of sequence
        else:
            if len(kmer) != k:
                continue
        
        # count occurrence of this kmer in sequence
        if kmer in kmers:
            kmers[kmer] += 1
        else:
            kmers[kmer] = 1
    
    return kmers

In [6]:
# the binary sequence
binary = "0001110100"

# get all 3mers in the binary sequence
kmers = get_kmer_count_from_sequence(binary, k=3, cyclic=False)

# return the kmers dictionary
kmers

{'000': 1,
 '001': 1,
 '011': 1,
 '111': 1,
 '110': 1,
 '101': 1,
 '010': 1,
 '100': 1}

In [7]:
def get_debruijn_edges_from_kmers(kmers):
    """
    Every possible (k-1)mer (n-1 suffix and prefix of kmers) is assigned
    to a node, and we connect one node to another if the (k-1)mer overlaps 
    another. Nodes are (k-1)mers, edges are kmers.
    """
    # store edges as tuples in a set
    edges = set()
    
    # compare each (k-1)mer
    for k1 in kmers:
        for k2 in kmers:
            if k1 != k2:            
                # if they overlap then add to edges
                if k1[1:] == k2[:-1]:
                    edges.add((k1[:-1], k2[:-1]))
                if k1[:-1] == k2[1:]:
                    edges.add((k2[:-1], k1[:-1]))

    return edges

In [8]:
# the binary sequence
binary = "0000110010111101"

# get all 4mers in the binary sequence
kmers = get_kmer_count_from_sequence(binary, k=4, cyclic=True)
print(kmers)

# get a set of edges for all 4-mers matching by n-1
edges = get_debruijn_edges_from_kmers(kmers)

# return edges
edges

{'0000': 1, '0001': 1, '0011': 1, '0110': 1, '1100': 1, '1001': 1, '0010': 1, '0101': 1, '1011': 1, '0111': 1, '1111': 1, '1110': 1, '1101': 1, '1010': 1, '0100': 1, '1000': 1}


{('000', '000'),
 ('000', '001'),
 ('001', '010'),
 ('001', '011'),
 ('010', '100'),
 ('010', '101'),
 ('011', '110'),
 ('011', '111'),
 ('100', '000'),
 ('100', '001'),
 ('101', '010'),
 ('101', '011'),
 ('110', '100'),
 ('110', '101'),
 ('111', '110'),
 ('111', '111')}

In [9]:
def plot_debruijn_graph(edges, width=500, height=500):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">", 
        vsize=25,
        vstyle={"stroke": "black", "stroke-width": 2, "fill": "none"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "black", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph

In [10]:
# print the cyclic binary string represented by the de Bruijn graph 
print(binary)

# plot the graph
plot_debruijn_graph(edges);

0000110010111101


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t04ee05828cf44c05bc08b245279759f3"> 000 001 010 011 100 101 110 111

In [11]:
def random_sequence(seqlen):
    "Generate a random DNA sequence of a given length "
    return "".join([random.choice("ACGT") for i in range(seqlen)])

In [12]:
# set a random seed 
random.seed(123)

# get a random genome sequence
genome1 = random_sequence(25)
genome1

'AGATGAATGGACCGGCCATATAAGT'

In [13]:
# not all possible kmers occur in this sequence, some occur multiple times.
kmers = get_kmer_count_from_sequence(genome1, k=3)
kmers

{'AGA': 1,
 'GAT': 1,
 'ATG': 2,
 'TGA': 1,
 'GAA': 1,
 'AAT': 1,
 'TGG': 1,
 'GGA': 1,
 'GAC': 1,
 'ACC': 1,
 'CCG': 1,
 'CGG': 1,
 'GGC': 1,
 'GCC': 1,
 'CCA': 1,
 'CAT': 1,
 'ATA': 2,
 'TAT': 1,
 'TAA': 1,
 'AAG': 1,
 'AGT': 1,
 'GTA': 1,
 'TAG': 1}

In [14]:
# edges of connected (k-1)mers for k=3 representing the db graph
edges = get_debruijn_edges_from_kmers(kmers)
edges

{('AA', 'AG'),
 ('AA', 'AT'),
 ('AC', 'CC'),
 ('AG', 'GA'),
 ('AG', 'GT'),
 ('AT', 'TA'),
 ('AT', 'TG'),
 ('CA', 'AT'),
 ('CC', 'CA'),
 ('CC', 'CG'),
 ('CG', 'GG'),
 ('GA', 'AA'),
 ('GA', 'AC'),
 ('GA', 'AT'),
 ('GC', 'CC'),
 ('GG', 'GA'),
 ('GG', 'GC'),
 ('GT', 'TA'),
 ('TA', 'AA'),
 ('TA', 'AG'),
 ('TA', 'AT'),
 ('TG', 'GA'),
 ('TG', 'GG')}

In [15]:
# get kmers
kmers = get_kmer_count_from_sequence(genome1, k=6, cyclic=True)

# get db graph
edges = get_debruijn_edges_from_kmers(kmers)

# plot db graph
plot_debruijn_graph(edges, width=600, height=400);

# print the true sequence
print("the true sequence: {}".format(genome1))

the true sequence: AGATGAATGGACCGGCCATATAAGT


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tab1a2f33fefc4970b3937e6352af6bc7"> AAGTA AATGG ACCGG AGATG AGTAG ATAAG ATATA ATGAA ATGGA CATAT CCATA CCGGC CGGCC GAATG GACCG GATGA GCCAT GGACC GGCCA GTAGA TAAGT TAGAT TATAA TGAAT TGGAC

In [16]:
# get kmers
kmers = get_kmer_count_from_sequence(genome1, k=6, cyclic=False)

# get db graph
edges = get_debruijn_edges_from_kmers(kmers)

# plot db graph
plot_debruijn_graph(edges, width=600, height=400);

# print the true sequence
print("the true sequence: {}".format(genome1))

the true sequence: AGATGAATGGACCGGCCATATAAGT


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta52332955f5345aca0189bf71cca4ed7"> AATGG ACCGG AGATG ATAAG ATATA ATGAA ATGGA CATAT CCATA CCGGC CGGCC GAATG GACCG GATGA GCCAT GGACC GGCCA TATAA TGAAT TGGAC

In [17]:
kmers = get_kmer_count_from_sequence(genome1, k=4, cyclic=False)
edges = get_debruijn_edges_from_kmers(kmers)
plot_debruijn_graph(edges, width=800, height=400);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="400.0px" viewBox="0 0 800.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t40a9e0885ca54809a736c7f7681f591b"> AAG AAT ACC AGA ATA ATG CAT CCA CCG CGG GAA GAC GAT GCC GGA GGC TAA TAT TGA TGG